# Installations and Imports

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import pandas as pd
import os
import re
import csv
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import AutoTokenizer,create_optimizer,TFAutoModel

In [ ]:
BATCH_SIZE=128
MAX_LENGTH=64

# Dataset Preparation

## Downloads

In [ ]:
!pip install aicrowd-cli

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 3.4 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.7 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 170 kB 45.8 MB/s 
     |████████████████████████████████| 214 kB 71.8 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 51 kB 5.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 23.2.1
    Uninstalling pyzmq-23.2.1:
      Successfully uninstalled pyzmq-23.2.1
  Attempting uninstall: python-slugify
    Found existing installation: python-slugify 6.1.2
    Uninstalling python-slugify-6.1.2:
 

In [ ]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/ww3_yLsfI7GJoSTmcpouplCpENYs09xfMMQg2O4kJ9k
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/ww3_yLsfI7GJoSTmcpouplCpENYs09xfMMQg2O4kJ9k'
API Key valid
Gitlab access token valid
Saved details successfully!


In [ ]:
!aicrowd dataset download -c esci-challenge-for-improving-product-search

product_catalogue-v0.3.csv.zip: 100% 328M/328M [00:28<00:00, 11.5MB/s]
sample_submission_public-v0.3.csv.zip: 100% 331k/331k [00:00<00:00, 339kB/s]
test_public-v0.3.csv.zip: 100% 394k/394k [00:01<00:00, 367kB/s]
train-v0.3.csv.zip: 100% 6.80M/6.80M [00:01<00:00, 3.46MB/s]
product_catalogue-v0.3.csv.zip: 100% 657M/657M [00:54<00:00, 12.0MB/s]
sample_submission_public-v0.3.csv.zip: 100% 812k/812k [00:01<00:00, 588kB/s]
test_public-v0.3.csv.zip: 100% 2.94M/2.94M [00:01<00:00, 1.63MB/s]
train-v0.3.csv.zip: 100% 19.8M/19.8M [00:03<00:00, 6.08MB/s]
product_catalogue-v0.3.csv.zip: 100% 657M/657M [00:55<00:00, 11.9MB/s]
sample_submission_public-v0.3.csv.zip: 100% 803k/803k [00:01<00:00, 583kB/s]
test_public-v0.3.csv.zip: 100% 2.94M/2.94M [00:01<00:00, 1.62MB/s]
train-v0.3.csv.zip: 100% 20.3M/20.3M [00:03<00:00, 6.19MB/s]


## Decompress and Copy

In [ ]:
!unzip "/content/train-v0.3.csv.zip" -d "/content/dataset/"

Archive:  /content/train-v0.3.csv.zip
  inflating: /content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv  


In [ ]:
!unzip "/content/product_catalogue-v0.3.csv.zip" -d "/content/dataset/"

Archive:  /content/product_catalogue-v0.3.csv.zip
  inflating: /content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv  



In [ ]:
!cp /content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv /content/drive/MyDrive/datasets/kdd_cup

In [ ]:
!cp /content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv /content/drive/MyDrive/datasets/kdd_cup

## Data Loading

In [ ]:
filepath_train='/content/drive/MyDrive/datasets/kdd_cup/train-v0.3_1.csv'

In [ ]:
filepath_catalogue='/content/drive/MyDrive/datasets/kdd_cup/product_catalogue-v0.3.csv'

In [ ]:
df_catalogue=pd.read_csv(filepath_catalogue)

In [ ]:
df_train=pd.read_csv(filepath_train)

In [ ]:
df_train.loc[df_train['query_locale'] == 'jp']

,query_id,query,query_locale,product_id,esci_label
572650,26004,0係,jp,B00FNJ1CTQ,substitute
572651,26004,0係,jp,B00FW60P84,irrelevant
572652,26004,0係,jp,B00GAYCNDM,substitute
572653,26004,0係,jp,B00MFBKQKG,substitute
572654,26004,0係,jp,B01G1YXU28,irrelevant
...,...,...,...,...,...
781739,33803,針なしほっちきす,jp,B08XGQ9RH7,substitute
781740,33803,針なしほっちきす,jp,B0987RGRF2,exact
781741,33803,針なしほっちきす,jp,B099NFJWP6,exact
781742,33803,針なしほっちきす,jp,B09F3B413J,exact


In [ ]:
df_catalogue

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),NaN,NaN,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,NaN,NaN,11 Degrees,NaN,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,NaN,NaN,11 Degrees,Negro (,es
...,...,...,...,...,...,...,...
1815211,B09FJYBVKS,スポーツブラ 揺れない ヨガウェア レディース トップス 通気性 ナイトブラ スポーツブラジ...,<p>【注意事項】</p><p>※商品内容以外のアクセサリー・小物等は付属しません.</p>...,シンプルのデザインで合わせやすいです。コート、半袖、シャツを合わせて、家で着てもいい、寝てい...,Doworspaw,ベージュ,jp
1815212,B09FTJ3S1N,ブラジャー レディース 補正ブラ Gabrioir 揺れない 大きいサイズ バストアップ美胸...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,パープル,jp
1815213,B09G72LDQZ,レディース ブラジャー 補正ブラ トップス Gabrioir 大きいサイズ 揺れない 脇高設...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,ベージュ,jp
1815214,B09GBGRTPB,ナイトブラ ノンワイヤーブラ ブラジャー レディース Gabrioir 大きいサイズ パット...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,パープル,jp


In [ ]:
def get_product_title(product_id):
  options=[]
  options.append(str(product_id))
  matched_row=df_catalogue.loc[df_catalogue['product_id'].isin(options)]
  #print(int(str(matched_row['product_title']).split("   ")[0]))
  return matched_row['product_title'][int(str(matched_row['product_title']).split("   ")[0])]

In [ ]:
get_product_title("B08N6NCKRX")### irrelevant

'面部保护 防雾 耐用 5个 透明面罩 健康盾牌 塑料面巾 完全可见 透气 可重复使用 成人 口罩 方便 打招呼 防护 (Clear)'

## Dataset Generation

In [ ]:
header = ['query', 'product', 'label']

with open('/content/drive/MyDrive/datasets/kdd_cup/csv_file_9.csv', 'w', encoding='UTF8') as f:
  
  # create the csv writer
  writer = csv.writer(f)

  # write the header
  writer.writerow(header)

  for i in range(781000,781636):
    
    data=[df_train[i:i+1]['query'][i],get_product_title(df_train[i:i+1]['product_id'][i]),df_train[i:i+1]['esci_label'][i]]
    # write the data
    writer.writerow(data)
    

In [ ]:
filepath="/content/drive/MyDrive/datasets/kdd_cup/csv_file_9.csv"
train_data=pd.read_csv(filepath)

In [ ]:
train_data

,query,product,label
0,ベビーマグ ちゃん 効果 なし,ピジョン Pigeon マグマグ スパウト 200ml 5ヵ月頃から,exact
1,ベビーマグ ちゃん 効果 なし,コンビ テテオ teteo マグベビー バルーン Neo,exact
2,ベビーマグ ちゃん 効果 なし,コンビ テテオ teteo パーティマグ 離乳ナビセット Neo マルチカラー 1個アソート...,exact
3,ベビーマグ ちゃん 効果 なし,レック アンパンマン ワンタッチ ストローマグ 300ml,irrelevant
4,ベビーマグ ちゃん 効果 なし,ピジョン Pigeon マグマグ セット 3ヵ月頃から 発達にあわせた4種類の飲み方トレーニング,substitute
...,...,...,...
631,肩なしブラジャー,スポーツブラ Lalaly【2枚組】 スポーツブラ 速乾 伸張 パット付き 脇高 はみ肉スッ...,substitute
632,肩なしブラジャー,2枚組スポーツブラ T-wilker 通気性 揺れない パッド入り ノンワイヤーブラ ナイト...,exact
633,肩なしブラジャー,（AQUIVER）肩ひも無し ブラ ストラップレス ブラジャー 肩ひもなし 背中見せ インナ...,exact
634,肩なしブラジャー,ずれにくい 6色6サイズ展開 レディース チューブトップ ブラ ブラジャー ナイトブラ ベア...,exact


## Data Preprocessing

In [ ]:
filepaths=["/content/drive/MyDrive/datasets/kdd_cup/csv_file_"+str(i)+".csv" for i in range(10)]
print(filepaths)

['/content/drive/MyDrive/datasets/kdd_cup/csv_file_0.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_1.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_2.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_3.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_4.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_5.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_6.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_7.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_8.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file_9.csv']


In [ ]:
dataset = load_dataset('csv', data_files=filepaths)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-040c829cfb31fc59/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'product', 'label'],
        num_rows: 731636
    })
})

In [ ]:
dataset['train'][0]

{'query': '# 2 pencils not sharpened',
 'product': 'Ticonderoga Beginner Pencils, Wood-Cased #2 HB Soft, With Eraser, Yellow, 12-Pack (13308)',
 'label': 'exact'}

In [ ]:
def get_label(label):
  if label=='exact':
    return 1.0
  elif label=='substitute':
    return 0.7
  elif label=='complement':
    return 0.5
  else:
    return 0.0

In [ ]:
model_id="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
def preprocess(dataset):
  if dataset['product']==None:
    dataset['product']=dataset['query']

  dataset['input_ids_query']=[]
  dataset['token_type_ids_query']=[]
  dataset['attention_mask_query']=[]

  dataset['input_ids_product']=[]
  dataset['token_type_ids_product']=[]
  dataset['attention_mask_product']=[]
  
  tokenized_output_query=tokenizer(dataset['query'],max_length=MAX_LENGTH,padding='max_length',truncation=True)
  tokenized_output_product=tokenizer(dataset['product'],max_length=MAX_LENGTH,padding='max_length',truncation=True)
  
  dataset['input_ids_query'].append(tokenized_output_query['input_ids'])
  dataset['token_type_ids_query'].append(tokenized_output_query['token_type_ids'])
  dataset['attention_mask_query'].append(tokenized_output_query['attention_mask'])
  
  dataset['input_ids_product'].append(tokenized_output_product['input_ids'])
  dataset['token_type_ids_product'].append(tokenized_output_product['token_type_ids'])
  dataset['attention_mask_product'].append(tokenized_output_product['attention_mask'])

  dataset['label']=get_label(dataset['label'])
  
  return dataset

In [ ]:
prep_dataset=dataset.map(preprocess)

  0%|          | 0/731636 [00:00<?, ?ex/s]

In [ ]:
prep_dataset['train'][21:22]

{'query': ['# 2 pencils not sharpened'],
 'product': ['BIC Evolution Cased Pencil, #2 Lead, Gray Barrel, 24-Count (PGEBP241-BLK)'],
 'label': [0.7],
 'input_ids_query': [[[101,
    1001,
    1016,
    14745,
    2015,
    2025,
    26694,
    102,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0]]],
 'token_type_ids_query': [[[0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    

In [ ]:
tf_dataset = prep_dataset["train"].to_tf_dataset(
    columns=['input_ids_query', 'token_type_ids_query', 'attention_mask_query','input_ids_product', 'token_type_ids_product', 'attention_mask_product', 'label'],
    shuffle=True, 
    batch_size=BATCH_SIZE,
)

In [ ]:
tf_dataset

<PrefetchDataset element_spec={'label': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'input_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'input_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None)}>

In [ ]:
for i in tf_dataset.take(1):
  print(i)

{'label': <tf.Tensor: shape=(128,), dtype=float32, numpy=
array([1. , 1. , 1. , 1. , 1. , 0. , 0.7, 1. , 0. , 0.5, 1. , 0.7, 0.7,
       0.7, 1. , 0. , 1. , 1. , 0.7, 0.7, 0.7, 0.7, 1. , 0. , 1. , 0.7,
       1. , 0.7, 0.7, 0.7, 1. , 1. , 1. , 0.7, 0.7, 0.7, 0.7, 0. , 0.7,
       0.5, 0. , 0.7, 1. , 0.7, 0.7, 1. , 0.7, 1. , 0. , 1. , 0.5, 1. ,
       0.7, 1. , 0. , 1. , 0.7, 0.7, 1. , 0. , 1. , 1. , 0.5, 1. , 0.7,
       0. , 1. , 1. , 1. , 1. , 0.5, 0.7, 1. , 0.7, 1. , 1. , 0.7, 1. ,
       0.7, 0.7, 0.7, 1. , 1. , 1. , 1. , 0.7, 1. , 1. , 0.7, 0.7, 0. ,
       0.7, 1. , 0.5, 0.7, 0.7, 1. , 0. , 1. , 1. , 1. , 1. , 1. , 0. ,
       0.7, 1. , 1. , 1. , 0.7, 1. , 1. , 1. , 1. , 1. , 0.7, 1. , 0. ,
       1. , 0. , 1. , 1. , 0.7, 0.7, 0.7, 1. , 1. , 1. , 0.7],
      dtype=float32)>, 'input_ids_query': <tf.Tensor: shape=(128, 1, 64), dtype=int64, numpy=
array([[[  101,  4942,  4313, ...,     0,     0,     0]],

       [[  101,  1713, 30260, ...,     0,     0,     0]],

       [[  101,   1

# Modeling

In [ ]:
model = TFAutoModel.from_pretrained(model_id)
model.summary()

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at sentence-transformers/multi-qa-MiniLM-L6-cos-v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  22713216  
                                                                 
Total params: 22,713,216
Trainable params: 22,713,216
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class SentenceTransformer(tf.keras.Model):
  def __init__(self,model):
    super(SentenceTransformer,self).__init__()
    self.model=model
    self.dense=Dense(1,activation='sigmoid')

  def compile(self,optimizer,loss_fn):
    super(SentenceTransformer,self).compile()
    self.optimizer=optimizer
    self.loss_fn=loss_fn
    self.loss_metric=tf.keras.metrics.Mean(name='loss')

  @property
  def metrics(self):
    return [self.loss_metric]
  
  def mean_pooling(self, model_output, attention_mask):
    token_embeddings = model_output[0]
    
    input_mask_expanded = tf.cast(
        tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
        tf.float32
    )
    return tf.math.reduce_sum(token_embeddings * input_mask_expanded, axis=1)/tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, tf.float32.max)
  
  def train_step(self,train_data):
    
    query={'input_ids':train_data['input_ids_query'][:,0,:],
           'token_type_ids':train_data['token_type_ids_query'][:,0,:],
           'attention_mask':train_data['attention_mask_query'][:,0,:]}

    product={'input_ids':train_data['input_ids_product'][:,0,:],
             'token_type_ids':train_data['token_type_ids_product'][:,0,:],
             'attention_mask':train_data['attention_mask_product'][:,0,:]}

    labels=train_data['label']

    with tf.GradientTape() as recorder:
      query_predictions=self.model(query)
      pred_query=self.mean_pooling(query_predictions,train_data['attention_mask_query'][:,0,:])
      
      product_predictions=self.model(product)
      pred_product=self.mean_pooling(product_predictions,train_data['attention_mask_product'][:,0,:])
      
      pred_concat=tf.concat([pred_query,pred_product,tf.abs(pred_query-pred_product)],axis=-1)
      
      predictions=self.dense(pred_concat)
      loss=self.loss_fn(labels,predictions)

      
    partial_derivatives = recorder.gradient(loss,self.model.trainable_weights)
    self.optimizer.apply_gradients(zip(partial_derivatives, self.model.trainable_weights))


    self.loss_metric.update_state(loss)
    
    return {'loss':self.loss_metric.result(),}

# Model Training

In [ ]:
stransformer=SentenceTransformer(model)
stransformer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5,),
    loss_fn=tf.keras.losses.BinaryCrossentropy(),
)

In [ ]:
EPOCHS=5
history=stransformer.fit(tf_dataset,epochs=EPOCHS,)

Epoch 1/5


1574/5716 [=======>......................] - ETA: 34:49 - loss: 0.4792

KeyboardInterrupt: ignored

In [ ]:
model_path='/content/drive/MyDrive/stransformer/stransformers.h5'

In [ ]:
#stransformer.model.save_weights(model_path)

In [ ]:
model.load_weights(model_path)

In [ ]:
def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  
  input_mask_expanded = tf.cast(
      tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
      tf.float32
  )
  return tf.math.reduce_sum(token_embeddings * input_mask_expanded, axis=1)/tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, tf.float32.max)

# Testing

## Create Embeddings

In [ ]:
filepath_catalogue='/content/drive/MyDrive/datasets/kdd_cup/product_catalogue-v0.3.csv'
df_catalogue=pd.read_csv(filepath_catalogue)

In [ ]:
df_catalogue['product_title'][1000000]

'【国内正規品】PeakDesign ピークデザイン シェルS SH-S-1'

In [ ]:
product_titles=[str(df_catalogue['product_title'][i]) for i in range(len(df_catalogue))]
#print(product_titles)

In [ ]:
print(len(product_titles))

1815216


In [ ]:
embeddings=[]

In [ ]:
INFERENCE_BATCH_SIZE=640
len(product_titles)//INFERENCE_BATCH_SIZE

2836

In [ ]:
for i in range(len(product_titles)//INFERENCE_BATCH_SIZE):
  tokenized_output=tokenizer(
      product_titles[INFERENCE_BATCH_SIZE*i:INFERENCE_BATCH_SIZE*(i+1)],max_length=MAX_LENGTH,padding='max_length',truncation=True,return_tensors="tf")
  model_output=model(tokenized_output)
  embedding=mean_pooling(model_output,tokenized_output['attention_mask'])
  embeddings.append(embedding)
  if i%100==0:
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [ ]:
embeddings

[<tf.Tensor: shape=(2560, 384), dtype=float32, numpy=
 array([[ 0.04175833,  0.4398206 ,  0.15303786, ..., -0.18008585,
         -0.71677995,  0.2654784 ],
        [-0.05360471, -0.13363229,  0.06446703, ..., -0.04043194,
         -0.5303095 ,  0.6396754 ],
        [-0.07222205,  0.568274  ,  0.04923436, ..., -0.3087162 ,
         -0.8591781 ,  0.75944847],
        ...,
        [ 0.2783488 ,  0.7277557 , -0.19541022, ...,  0.25403762,
         -0.77744997,  0.88280714],
        [-0.11515236,  0.44915688, -0.32512447, ...,  0.15528241,
         -0.28903574, -0.01429977],
        [-0.71291393, -0.30155414,  0.60434943, ..., -0.52819324,
         -0.06169818, -0.42003578]], dtype=float32)>,
 <tf.Tensor: shape=(2560, 384), dtype=float32, numpy=
 array([[-0.25033033, -0.36277884,  0.04568261, ..., -0.61685425,
         -0.22854248,  0.0749432 ],
        [ 0.31579307,  0.3078013 , -0.15302154, ..., -0.0205166 ,
         -0.24058373, -0.1389403 ],
        [ 0.34076113,  0.09088283, -0.0992058

In [ ]:
#np.savez_compressed('embeddings.npz', embeddings)
#np.savez_compressed('product_titles.npz',product_titles)

In [ ]:
!cp '/content/embeddings.npz' '/content/drive/MyDrive/stransformer/'
!cp '/content/product_titles.npz' '/content/drive/MyDrive/stransformer/'

## Use Embeddings

In [ ]:
loaded_embedding=np.load('/content/drive/MyDrive/stransformer/embeddings.npz')
loaded_embedding_array=np.array(loaded_embedding['arr_0'])

In [ ]:
loaded_titles=np.load('/content/drive/MyDrive/stransformer/product_titles.npz')
loaded_titles_array=np.array(loaded_titles['arr_0'])

In [ ]:
loaded_embedding_array.shape

(2836, 640, 384)

In [ ]:
loaded_embedding_array=loaded_embedding_array.reshape(-1,loaded_embedding_array.shape[2])
print(loaded_embedding_array.shape)

(1815040, 384)


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
inputs = tokenizer(["針なしほっちきす"],max_length=MAX_LENGTH,padding='max_length',truncation=True,return_tensors="tf")

logits = model(**inputs)
out_embedding=mean_pooling(logits,inputs['attention_mask'])
print(out_embedding.shape)

(1, 384)


In [ ]:
u_dot_v=np.matmul(loaded_embedding_array,(np.array(out_embedding).T))
print(u_dot_v.shape)

(1815040, 1)


In [ ]:
u_magnitude=np.sqrt(np.sum(loaded_embedding_array*loaded_embedding_array,axis=-1))
print(u_magnitude.shape)
print(u_magnitude)

(1815040,)
[11.215763   9.297123  10.870735  ...  5.05177    7.0463676  4.7366333]


In [ ]:
v_magnitude=np.sqrt(np.sum(out_embedding*out_embedding,axis=-1))
print(v_magnitude.shape)
print(v_magnitude)

(1,)
[6.5474906]


In [ ]:
cosine_similarity=u_dot_v.T/(u_magnitude*v_magnitude)
print(cosine_similarity)

[[0.24800435 0.33141387 0.11507133 ... 0.12804486 0.35886842 0.08012675]]


In [ ]:
sorted_indices=np.argsort(cosine_similarity,axis=-1)
print(sorted_indices)

[[  83865 1386827  779273 ... 1811568 1139688 1803600]]


In [ ]:
for i in range(25):
  print(i,loaded_titles_array[sorted_indices[:,len(sorted_indices[0])-i-1]])

0 ['知りすぎたのね']
1 ['リバウンドしない収納']
2 ['ヴェノム (吹替版)']
3 ['ヴェノム (字幕版)']
4 ['メルちゃん おせわパーツ おしょくじ&はみがきセット']
5 ['スウィングしなけりゃ意味ないね']
6 ['スウィングしなけりゃ意味ないね']
7 ['スウィングしなけりゃ意味ないね']
8 ['スウィングしなけりゃ意味ないね']
9 ['スイングしなけりゃ意味ないね']
10 ['スウィングしなけりゃ意味ないね']
11 ['スイングしなけりゃ意味ないね']
12 ['スウィングしなけりゃ意味ないね']
13 ['スウィングしなけりゃ意味ないね']
14 ['スイングしなけりゃ意味ないね']
15 ['スウィングしなけりゃ意味ないね']
16 ['すみっコぐらしの英会話']
17 ['映画あたしンち']
18 ['クラリネットピース スウィングしなけゃ意味ないね']
19 ['すみっコぐらしのお弁当']
20 ['嫁入り道具の花ふきん']
21 ['ビジュアルヌード・ポーズＢＯＯＫ ａｃｔ松岡ちな']
22 ['Best of Ramin Djawadi']
23 ['CHARMINER 0045']
24 ['キズナアイ 1/7 完成品フィギュア']


In [ ]:
small_dataset = dataset.filter(lambda example: example["query"].startswith("A"))
print(len(small_dataset))